In [228]:
import math
import json
import pickle
import random
from gensim import models
from gensim import matutils
import logging
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import *
from time import time
from nltk.tokenize import sent_tokenize
import glob
import argparse
import os
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import *
from sklearn.cluster import *
from scipy import sparse
path2files="../dataset/"
path2buisness=path2files+"yelp_academic_dataset_business.json"
path2reviews=path2files+"yelp_academic_dataset_review.json"

In [230]:
K = 30
vectorizer = TfidfVectorizer(max_df=0.5, max_features=10000,
                                     min_df=2, stop_words='english',
                                     use_idf=True)
text = []
cat_list = glob.glob("categories/*")
cat_size = len(cat_list)
print("Total Categories are : ",cat_size)
sample_size = 30
#sample_size = min(30, cat_size)
cat_sample = []
cat_names = []
cat_sample = sorted( random.sample(range(cat_size), sample_size) )
#print (cat_sample)
count = 0
for i, item in enumerate(cat_list):
    if i == cat_sample[count]:
        li =  item.split('/')
        cuisine_name = li[-1]
        cat_names.append(cuisine_name[:-4].replace("_"," "))
        with open ( item ) as f:
            text.append(f.read().replace("\n", " "))
        count = count + 1
print(cat_names)       

Total Categories are :  56
['Donuts', 'Indonesian', 'Chicken Wings', 'Tours', 'Shanghainese', 'Buffets', 'Active Life', 'Arabian', 'Himalayan-Nepalese', 'Venezuelan', 'Music Venues', 'Tex-Mex', 'Diners', 'Tea Rooms', 'Scandinavian', 'Steakhouses', 'Fish & Chips', 'Do-It-Yourself Food', 'Hawaiian', 'Karaoke', 'Afghan', 'Juice Bars & Smoothies', 'Food', 'African', 'Grocery', 'Hotels', 'Food Stands', 'Portuguese', 'Local Flavor', 'Car Wash']


In [231]:
t0 = time()
print("Extracting features from the training dataset using a sparse vectorizer")
X = vectorizer.fit_transform(text)
print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)

Extracting features from the training dataset using a sparse vectorizer
done in 17.540260s
n_samples: 30, n_features: 10000


In [232]:
corpus = matutils.Sparse2Corpus(X,  documents_columns=False)
X[0]

<1x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 861 stored elements in Compressed Sparse Row format>

In [233]:
true_k = 30
mIter = 400
model = MiniBatchKMeans(n_clusters = true_k, init="k-means++", max_iter = mIter, batch_size = 50)
model.fit(X)

MiniBatchKMeans(batch_size=50, compute_labels=True, init='k-means++',
        init_size=None, max_iter=400, max_no_improvement=10, n_clusters=30,
        n_init=3, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=0)

In [234]:
model.labels_

array([18, 12, 11, 17, 13,  6,  5, 26, 16,  7, 24,  3, 25, 23, 10, 28, 19,
        8,  4, 29,  2, 27,  1, 21, 22, 15, 20,  9, 14,  0], dtype=int32)

In [235]:
print("Top terms per cluster")
order_centroids = model.cluster_centers_.argsort()[:,::-1]

Top terms per cluster


In [244]:
terms = vectorizer.get_feature_names() ## return all terms from the corpus
kmeans_out = []
for i in range(true_k):
    print("Cluster : %d"%i)
    kmeans_out.append("Cluster : %d"%i)
    for ind in order_centroids[i,:30]:
        print(" %s"%terms[ind])
        kmeans_out.append(" %s"%terms[ind])
    print()    
    with open ("kmeans_topics.txt", 'w' ) as f:
        f.write('\n'.join(kmeans_out))

Cluster : 0
 campfire
 grits
 carolina
 hickory
 shoestring
 cubano
 caprese
 fuel
 chandelier
 tunnel
 cuban
 slider
 barbeque
 cobbler
 merchandise
 pigs
 gallon
 browse
 vehicle
 stations
 attendant
 cave
 grit
 gingerbread
 rudy
 oklahoma
 exceed
 condo
 racing
 drying

Cluster : 1
 bagel
 bagels
 lux
 crepe
 gelato
 crepes
 macarons
 croissant
 boba
 venetian
 macaroons
 quiche
 croissants
 peaks
 bouchon
 mocha
 chai
 panini
 brewery
 pho
 bj
 wildflower
 macaron
 scones
 chompie
 slider
 baguette
 palazzo
 lgo
 einstein

Cluster : 2
 kabob
 kabobs
 kabul
 afghan
 naan
 afghani
 gyros
 masala
 basmati
 flaming
 tikka
 pakistani
 boti
 halal
 samosas
 tzatziki
 persian
 baklava
 baba
 raisins
 karahi
 arabic
 kofta
 lentil
 samosa
 falafel
 shwarma
 shish
 kebab
 shawarma

Cluster : 3
 margaritas
 fajitas
 enchiladas
 tejas
 tex
 mex
 burritos
 cornbread
 guac
 enchilada
 richardson
 abuelo
 chambord
 abuelos
 carnitas
 eldorado
 barbacoa
 salsas
 chipotles
 relleno
 tb
 nacho
 do

In [247]:
terms[2000]

'compelled'

In [258]:
kmean_cluster = model.cluster_centers_.argsort()[:,::-1]

In [269]:
kmw = model.cluster_centers_[:,::-1]

In [271]:
kmw.shape

(30, 10000)

In [257]:
kmw * kmean_cluster

array([[  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  3.50992253,   1.32521954,  13.66369786, ...,   5.6576996 ,
          2.02150765,  12.15893584],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       ..., 
       [ 34.80774815,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  4.99088273,   1.63653719,   7.58020603, ...,   4.9515285 ,
         23.92317585,   6.4491891 ],
       [ 12.02704616,  15.37936799,  23.5484112 , ...,   0.        ,
          0.        ,   0.        ]])

In [261]:
kmw[2]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [238]:
print("n_samples: %d, n_features: %d" % kmean_cluster.shape)
n_samples = kmean_cluster.shape[0]

n_samples: 30, n_features: 10000


In [239]:
cosine_similarities = np.zeros((n_samples, n_samples))
for i in range(0,n_samples):
    for j in range(0,n_samples):
        A = sparse.csr_matrix(kmean_cluster[i])
        B = sparse.csr_matrix(kmean_cluster[j])
        cosine_similarities[i][j] = cosine_similarity(A,B)[0]

In [240]:
kmdf = pd.DataFrame(cosine_similarities)
kmdf.to_csv("sim_matrix_cluster_idf.csv", sep=',', header=False,index=False)

In [241]:
import csv
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import cm as cm
def read_labels():
    cuisine_list = []
    with open(LABELS_FILE, 'r') as f:
        data = f.readlines()
        for line in data:
            words = line.rstrip('\n')
            cuisine_list.append(words)
    print(cuisine_list)
    return cuisine_list

def read_simMatrix_csv(mf):
    matrixdf = pd.read_csv(mf, header=None)
    print(matrixdf[0])
    return matrixdf


def plot_sim_matrix(title, labels, df, mpng,ccode):
    fig, ax = plt.subplots(figsize = (20,20))
    cmap = cm.get_cmap(ccode)
    cax = ax.matshow(df, interpolation='nearest', cmap = cmap)
    ax.grid(False)
    plt.title(title,y=1.18)
    plt.xticks(range(30), labels, rotation = 90)
    plt.yticks(range(30), labels)
    fig.colorbar(cax, ticks=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 
                             0.6, 0.65, 0.7,0.75, 0.8,0.85, 0.9,0.95, 1.0])
    fig.savefig(mpng)
    plt.close(fig)
    return

In [243]:
kmeans = "sim_matrix_cluster_idf.csv"
kmeansf = read_simMatrix_csv(kmeans)
plot_sim_matrix('KMeans IDF Similarity Matrix',cat_names, kmeansf,"kmeans_idf.png","Greens")

0     1.000000
1     0.745628
2     0.978830
3     0.741300
4     0.744545
5     0.833843
6     0.751986
7     0.972668
8     0.981333
9     0.986711
10    0.957967
11    0.739402
12    0.987905
13    0.988301
14    0.988119
15    0.736918
16    0.987190
17    0.982304
18    0.958817
19    0.776735
20    0.888138
21    0.892573
22    0.745836
23    0.879517
24    0.737900
25    0.744347
26    0.982131
27    0.767975
28    0.752268
29    0.880069
Name: 0, dtype: float64
